In [2]:
import geopandas as gpd
import pandas as pd
import folium
import pickle
import osmnx as ox


In [3]:
gdf = gpd.read_file("../data/data_all_gpd_v2.shp")
gdf.head()

,started_at,ended_at,duration,st_st_id,st_st_name,st_st_desc,st_st_lat,st_st_lon,en_st_id,en_st_name,en_st_desc,en_st_lat,en_st_lon,minutes,id,total_dist,waytypes_d,waytypes_s,geometry
0,2022-07-02 08:39:02.618000+00:00,2022-07-02 08:47:54.958000+00:00,532,408,Tøyen skole,forsiden av skolebygget,59.914943,10.773977,448,Oslo Plaza,ved rundkjøringen,59.912183,10.754434,8,6044,1502.2,0 3 7;3 13 2;13 20 3;20 50 2;50 82 3;82 91 7;,v:3.0 d:805.9 a:53.65;v:2.0 d:638.3 a:42.49;v:...,"LINESTRING (10.77395 59.91491, 10.77397 59.914..."
1,2022-07-02 08:39:04.505000+00:00,2022-07-02 08:48:36.381000+00:00,571,397,Storo Storsenter,langs Vitaminveien,59.946710,10.773805,421,Alexander Kiellands Plass,langs Maridalsveien,59.928067,10.751203,9,6045,2701.3,0 2 7;2 86 2;86 94 7;94 111 6;111 112 7;112 11...,v:2.0 d:1435.3 a:53.14;v:3.0 d:805.4 a:29.82;v...,"LINESTRING (10.77378 59.94668, 10.77369 59.946..."
2,2022-07-02 08:39:22.379000+00:00,2022-07-02 08:55:10.095000+00:00,947,601,Rolf Wickstrøms vei,ved Bergrådveien,59.953411,10.757724,2309,Ulven Torg,"innenfor byggegjerdene, sør for Ulvenveien",59.924721,10.810727,15,6047,5583.8,0 4 7;4 83 6;83 93 7;93 141 6;141 142 7;142 16...,v:6.0 d:2775.3 a:49.7;v:3.0 d:1226.1 a:21.96;v...,"LINESTRING (10.75776 59.95342, 10.75783 59.953..."
3,2022-07-02 08:40:20.905000+00:00,2022-07-02 09:03:56.044000+00:00,1415,464,Sukkerbiten,ved gangbroen,59.905124,10.753764,561,Kirkeveien,ved Gørbitz' gate,59.933480,10.726294,23,6052,4430.6,0 3 7;3 7 4;7 8 7;8 18 2;18 42 1;42 139 2;139 ...,v:2.0 d:2641.3 a:59.61;v:3.0 d:841.5 a:18.99;v...,"LINESTRING (10.75380 59.90512, 10.75386 59.905..."
4,2022-07-02 08:40:45.470000+00:00,2022-07-02 08:50:08.635000+00:00,563,519,Thomas Heftyes plass,ved Bygdøy allé,59.918578,10.703900,473,Majorstuveien,langs Maries gate,59.926697,10.716195,9,6053,1409.8,0 3 7;3 30 2;30 54 1;54 64 3;,v:1.0 d:657.1 a:46.61;v:2.0 d:427.4 a:30.32;v:...,"LINESTRING (10.70389 59.91857, 10.70347 59.918..."


In [4]:
len(gdf)

25632

In [5]:
st_coor = [[row['st_st_lat'], row['st_st_lon']] for _, row in gdf.iterrows()]
en_coor = [[row['en_st_lat'], row['en_st_lon']] for _, row in gdf.iterrows()]
agg_coor = st_coor
agg_coor.append(en_coor)

In [6]:
cood_df = pd.DataFrame()
st_lat = [row['st_st_lat'] for _, row in gdf.iterrows()]
en_lat = [row['en_st_lat'] for _, row in gdf.iterrows()]
st_lon = [row['st_st_lon'] for _, row in gdf.iterrows()]
en_lon = [row['en_st_lon'] for _, row in gdf.iterrows()]

lat = st_lat
lat.extend(en_lat)

lon = st_lon
lon.extend(en_lon)

cood_df['lon'] = lon
cood_df['lat'] = lat

In [7]:
cood_df = cood_df.drop_duplicates()
cood_df.head()

,lon,lat
0,10.773977,59.914943
1,10.773805,59.946710
2,10.757724,59.953411
3,10.753764,59.905124
4,10.703900,59.918578
...,...,...
3930,10.720789,59.926894
3934,10.786856,59.908576
4031,10.751551,59.915983
5178,10.735113,59.911776


In [ ]:
cood_df.to_csv('../data/coordinates.csv', index=False)

In [8]:
with open('../data/bike_graph.pkl', 'rb') as f:
     G_bike = pickle.load(f)
     
gdf_edges = ox.graph_to_gdfs(G_bike, nodes=False, edges=True)

In [9]:
all_data = gpd.read_file("../data/data_all_gpd.shp")
all_data = all_data['geometry']
all_data = all_data.drop_duplicates()

In [ ]:
m = folium.Map(location=[59.916823,10.747777], zoom_start=13)

for index, row in cood_df.iterrows():
    folium.Marker([row['lat'], row['lon']]).add_to(m)

#This code add all the used road but then the vs code gonna crash out
folium.Choropleth(
    gdf,
    line_weight=3,
    line_color="blue"
).add_to(m)
    
folium.Choropleth(
    gdf_edges,
    line_weight=1,
    line_color="red"
).add_to(m)

m